# Ch 5 - Deep Learning for Computer Vision

This chapter introduces convolutional neural networks, also known as convnets, a type of deep-learning model almost universally used in computer vision applications. 

## 5.1 Introduction to Convnets

Going to use the same MNIST digits dataset as in Ch 2. 

In [1]:
from keras import layers
from keras import models

Using TensorFlow backend.


### Instantiating a Small Convnet

This is what a basic convnet looks like. It is a stack of Conv2D and MaxPooling2D layers. 

The convnet takes input tensors of shape (image_height, image_width, image_channels). In this case, we'll configure the convnet to process inputs of size (28, 28, 1), which is the format of MNIST  images. 

In [2]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', \
                        input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

#### Architecture of the Convnet:

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


The output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels).

The width and height dimensions tend to shrink as you go deeper in the network.

The number of channels is controlled by the first argument passed to the Conv2D layers (32 or 64).

#### Adding a Classifier on Top of the Convnet

The next step is to feed the last output tensor (of shape (3, 3, 64)) into a densely connected classifier network like those you're already familiar with: a stack of Dense layers.

These classifiers process vectors, which are 1D, whereas the current output is a 3D tensor. First we need to flatten the 3D outputs to a 1D, and then add a few Dense layers on top.

In [4]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


We’ll do 10-way classification, using a final layer with 10 outputs and a softmax activation.

Here’s what the network looks like now:

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

The (3, 3, 64) outputs are flattened into vectors of shape (576, ) before going through two Dense layers.

#### Training the Convnet on MNIST Images

In [6]:
from keras.datasets import mnist
from keras.utils import to_categorical

In [7]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [8]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [9]:
model.compile(optimizer='rmsprop',\
             loss='categorical_crossentropy',\
             metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [10]:
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 33s 542us/step - loss: 0.1727 - acc: 0.9448
Epoch 2/5
60000/60000 [==============================] - 33s 546us/step - loss: 0.0478 - acc: 0.9850
Epoch 3/5
60000/60000 [==============================] - 34s 563us/step - loss: 0.0334 - acc: 0.9894
Epoch 4/5
60000/60000 [==============================] - 34s 560us/step - loss: 0.0249 - acc: 0.9921
Epoch 5/5
60000/60000 [==============================] - 34s 560us/step - loss: 0.0191 - acc: 0.9939


#### Accuracy:

In [11]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

test_acc

10000/10000 [==============================] - 2s 207us/step


0.9901

Whereas the densely connected network from chapter 2 had a test accuracy of 97.8%, the basic convnet has a test accuracy of 99.1%: we decreased the error rate by 68% (relative)!

### 5.1.1 The Convolution Operation

The fundamental difference between a densely connected layer and a convolution layer is this: 

- Dense layers learn global patterns in their input feature space (for example, for a MNIST digit, patterns involving all pixels)

- Convolution layers learn local patterns (see figure 5.1): in the case of images, patterns found in small 2D windows of the inputs. In the previous example, these windows were all 3 × 3.

![4](Images/05_01.jpg)


This gives convnets two interesting properties:

- **The patters they learn are translation invariant.** After learning a certain pattern in the lower-right corner of a picture, a convnet can recognize it anywhere: for example, in the upper-left corner. A densely connected network would have to learn the pattern anew if it appeared at a new location. This makes convnets data efficient when processing images (because the visual world is fundamentally translation invariant): they need fewer training samples to learn representations that have generalization power.


- **They can learn spacial hierarchies of patterns.** A first convolution layer will learn small local patterns such as edges, a second convolution layer will learn larger patterns made of the features of the first layers, and so on. This allows convnets to efficiently learn increasingly complex and abstract visual concepts (because the visual world is fundamentally spatially hierarchical).


![cat](Images/05_02.jpg)




Convolutions operate over 3D tensors, called **feature maps**, with two spatial axes (**height** and **width**) as well as a **depth axis** (also called the **channels axis**). 

For an RGB image, the dimension of the depth axis is 3, because the image has three color channels: red, green, and blue. For a black-and-white picture, like the MNIST digits, the depth is 1 (levels of gray).

The convolution operation extracts patches from its input feature map and applies the same transformation to all of these patches, producing an **output feature map**. This output feature map is still a 3D tensor: it has a width and a height. Its depth can be arbitrary, because the output depth is a parameter of the layer, and the different channels in that depth axis no longer stand for specific colors as in RGB input; rather, they stand for **filters**. 

Filters encode specific aspects of the input data: at a high level, a single filter could encode the concept “presence of a face in the input,” for instance.

In the MNIST example, the first convolution layer takes a feature map of size (28, 28, 1) and outputs a feature map of size (26, 26, 32): it computes 32 filters over its input. Each of these 32 output channels contains a 26 × 26 grid of values, which is a **response map** of the filter over the input, indicating the response of that filter pattern at different locations in the input (see figure 5.3). That is what the term **feature map** means: every dimension in the depth axis is a feature (or filter), and the 2D tensor output [ :, :, n ] is the 2D spatial map of the response of this filter over the input.


![MNIST](Images/05_03.jpg)

Convolutions are defined by two key parameters:

- **Size of the patches extracted from the inputs**: These are typically 3 × 3 or 5 × 5. In the example, they were 3 × 3, which is a common choice.


- **Depth of the output feature map**: The number of filters computed by the convolution. The example started with a depth of 32 and ended with a depth of 64.